<a href="https://colab.research.google.com/github/GarettGazay/ai_projects/blob/master/Back_to_BART.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Original prompt:

please write a bart-base script that trains a model using BartForConditionalGeneration, Trainer, Seq2SeqTrainingArguments that has f1 and accuracy that uses inputs and targets in the following format.

input = ['2023-02-17 08:20 -121.8927307_37.27342606 -121.933504_37.313512 422032 SPNSTR 300.52 7.63', '2023-02-17 18:15 -121.93733_37.3272 -121.9516678_37.38991928 388574 BSTR 438.4 9.6',]

target = ['2023-02-17 04:45 -121.8577499_37.30191803 -121.8113861_37.30517197 SC103 388477 WCH 110.7 6.98', '2023-02-17 05:00 -121.8577499_37.30191803 -121.8113861_37.30517197 SC206 388472 WCH 82.63 12.77',]

In [1]:
!pip install transformers datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 57.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 42.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 103.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 78.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 26.1 MB/s eta 0:00:00


In [2]:
import pandas as pd
days = pd.read_csv('days.csv')

In [3]:
inputs = days['input']
inputs.tail()


413    ['2023-03-09 09:30 -121.9314804_37.31127167 -1...
414    ['2023-03-10 14:00 -122.0738525_37.38199615 -1...
415    ['2023-03-11 09:45 -121.811386_37.305172 -121....
416    ['2023-03-13 15:00 -121.8113861_37.30517197 -1...
417    ['2023-03-14 05:30 -121.9642258_37.29271698 -1...
Name: input, dtype: object

In [4]:
targets = days['target']
targets.tail()

413    ['2023-03-09 04:30 -121.8703613_37.4176445 -12...
414    ['2023-03-10 04:45 -121.9337082_37.31555939 -1...
415    ['2023-03-11 04:15 -121.8491516_37.32674408 -1...
416    ['2023-03-13 04:45 -121.9533844_37.29909897 -1...
417    ['2023-03-14 04:30 -122.1133575_37.4191246 -12...
Name: target, dtype: object

In [5]:
test_input = eval(inputs[396])
test_input

['2023-02-18 17:15 -121.8120956_37.30604935 -121.8320541_37.29574966 390537 WCH  25.09 2.26',
 '2023-02-18 04:15 -121.8182449_37.31994247 -121.8480759_37.36037445 482719 WCH  104.47 4.21',
 '2023-02-18 12:15 -121.8653641_37.38121033 -121.8461533_37.30810165 481655 WCH  112.33 7.7',
 '2023-02-18 13:00 -122.1483078_37.47801208 -122.1578217_37.45035553 388482 BWC  64.04 3.51',
 '2023-02-18 07:45 -121.8388824_37.35326004 -121.8269119_37.25194168 462296 SPNWC  167.84 10.71',
 '2023-02-18 13:45 -121.9790344_37.32246399 -121.9678116_37.3220253 388994 WCH  22.05 0.91',
 '2023-02-18 09:00 -121.931572_37.3112793 -121.8949127_37.32189178 390529 STR  126.17 2.74',
 '2023-02-18 08:45 -122.1578217_37.45035553 -122.1483078_37.47801208 388482 BWC  164.04 3.51',
 '2023-02-18 05:00 -121.8912888_37.31449127 -121.9314804_37.31127167 433433 WCH  104.34 4.15',
 '2023-02-18 05:30 -121.8949127_37.32189178 -121.931572_37.3112793 390529 STR  201.16 2.74',
 '2023-02-18 08:15 -121.8480759_37.36037445 -121.8182449

In [6]:
test_target = eval(targets[396])
test_target

['2023-02-18 04:15 -121.8134232_37.35033035 -121.8272247_37.31860733 SC106 482719 WCH  104.47 4.21',
 '2023-02-18 04:30 -121.8134232_37.35033035 -121.8272247_37.31860733 SC203 472150 STR  204.83 4.37',
 '2023-02-18 05:00 -121.8134232_37.35033035 -121.8272247_37.31860733 SC102 390534 WCH  103.3 3.69',
 '2023-02-18 05:00 -121.8134232_37.35033035 -121.8272247_37.31860733 SC102 433433 WCH  104.34 4.15',
 '2023-02-18 05:15 -121.8134232_37.35033035 -121.8272247_37.31860733 SC106 439288 WCH  105.31 4.58',
 '2023-02-18 05:30 -121.8134232_37.35033035 -121.8272247_37.31860733 SC203 390529 STR  201.16 2.74',
 '2023-02-18 05:30 -121.8134232_37.35033035 -121.8272247_37.31860733 SC106 388496 WCH  61.48 3.37',
 '2023-02-18 05:45 -121.8134232_37.35033035 -121.8272247_37.31860733 SC102 388574 BWC  210.48 15.12',
 '2023-02-18 05:45 -121.8134232_37.35033035 -121.8272247_37.31860733 SC106 388896 WCH  101.25 2.78',
 '2023-02-18 06:00 -121.8134232_37.35033035 -121.8272247_37.31860733 SC106 493371 WCH  101.2

In [7]:
len(test_input), len(test_target)

(82, 82)

In [8]:
with open('custom_vocab.txt','r') as file:
  custom_vocab = file.read().splitlines()
print(custom_vocab)

['106.16', '201.075', '-121.81915_37.312841', '49.115', '10.25946', '99.005', '37.02638245', '-121.8324966_37.29072189', '143.04', '07:12:06', '-121.9623947_37.30070877', '2022-07-28', '87.28', '100.175', '463427', '17.056019', '-121.9822311_37.34886169', '224.72', '249.72', '37.32243729', '37.33909225', '1.050117', '487678', '4.679546', '32.015', '-122.26309_37.82153', '2023-01-05', '-121.864686_37.24755', '37.3783992', '392.2', '24.59', '1.473892', '116.2625', '37.40153122', '42.3425', '8.775625', '440198', '37.331201', '463942', '51.1', '-121.9719696_37.22613144', '227.625', '-122.079144_37.382993', '388993', '224.0025', '109.44', '-118.418475_34.183884', '16:40:00', '-122.177103_37.434279', '36.425', '139.0125', '2022-11-21', '2022-02-14', '11.410861', '490398', '-121.9719772_37.22615433', '17.244915', '47.54', '13.189225', '390530', '65.9825', '37.31127167', '-121.8805285_37.3401237', '422833', '98.5325', '37.393067', '2.186605', '32.009937', '37.343521', '41.9825', '11:30:00', '2

With full dataset

In [9]:
# get the inputs and targets into the right format
inputs = [eval(x) for x in inputs]
targets = [eval(x) for x in targets]

inputs = [x for i, x in enumerate(inputs) if x or print(f'Empty array found in inputs at index {i} and left out')]
targets = [x for i, x in enumerate(targets) if x or print(f'Empty array found in targets at index {i} and left out')]

In [10]:
split_idx = int(0.8 * len(inputs))

train_inputs, train_targets = inputs[:split_idx], targets[:split_idx]
val_inputs, val_targets = inputs[split_idx:], targets[split_idx:]

In [11]:
# Create dataset list of tuples for batch_encode_plus => dataloader
train_inputs_targets = []
for input_, target_ in zip(train_inputs, train_targets):
  train_inputs_targets.append( (input_,target_) )
  
val_inputs_targets = []
for input_, target_ in zip(val_inputs, val_targets):
  val_inputs_targets.append( (input_,target_) )

In [12]:
import torch
from torch.utils.data import DataLoader
from transformers import BartTokenizer, BartForConditionalGeneration, BartConfig

# define your input and target data
input_data = [['2023-02-18 04:15 -121.8134232_37.35033035 -121.8272247_37.31860733 SC106 482719 WCH  104.47 4.21', '2023-02-18 04:30 -121.8134232_37.35033035 -121.8272247_37.31860733 SC203 472150 STR  204.83 4.37'], ['2023-02-18 05:15 -121.8134232_37.35033035 -121.8272247_37.31860733 SC106 482719 WCH  104.47 4.21', '2023-02-18 05:30 -121.8134232_37.35033035 -121.8272247_37.31860733 SC203 472150 STR  204.83 4.37']]
target_data = [['2023-02-18 04:15 -121.8134232_37.35033035 -121.8272247_37.31860733 SC106 482719 WCH  104.47 4.21', '2023-02-18 04:30 -121.8134232_37.35033035 -121.8272247_37.31860733 SC203 472150 STR  204.83 4.37'], ['2023-02-18 05:15 -121.8134232_37.35033035 -121.8272247_37.31860733 SC106 482719 WCH  104.47 4.21', '2023-02-18 05:30 -121.8134232_37.35033035 -121.8272247_37.31860733 SC203 472150 STR  204.83 4.37']]

# initialize the tokenizer and collator
tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')
collator = lambda x: {'input_ids': tokenizer(x[0], padding=True, truncation=True, return_tensors='pt')['input_ids'], 'decoder_input_ids': tokenizer(x[1], padding=True, truncation=True, return_tensors='pt')['input_ids'], 'decoder_attention_mask': tokenizer(x[1], padding=True, truncation=True, return_tensors='pt')['attention_mask'], 'labels': tokenizer(x[1], padding=True, truncation=True, return_tensors='pt')['input_ids']}

# define your model and optimizer
config = BartConfig.from_pretrained('facebook/bart-base')
model = BartForConditionalGeneration.from_pretrained('facebook/bart-base', config=config)
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

# define your training data
train_data = list(zip(input_data, target_data))

# define your dataloader
train_dataloader = DataLoader(train_data, batch_size=2, shuffle=True, collate_fn=collator)

# train your model on multiple batches
for epoch in range(10):
  for batch in train_dataloader:
    optimizer.zero_grad()
    input_ids = batch['input_ids'].squeeze(1)
    attention_mask = input_ids != tokenizer.pad_token_id
    decoder_input_ids = batch['decoder_input_ids'].squeeze(1)
    decoder_attention_mask = batch['decoder_attention_mask'].squeeze(1)
    labels = batch['labels'].squeeze(1)

    outputs = model(input_ids=input_ids, attention_mask=attention_mask, decoder_input_ids=decoder_input_ids, decoder_attention_mask=decoder_attention_mask, labels=labels)

    loss = outputs.loss
    loss.backward()
    optimizer.step()

    print(f"Epoch: {epoch+1} | Loss: {loss.item()}")



Epoch: 1 | Loss: 17.59856414794922
Epoch: 2 | Loss: 7.128303527832031
Epoch: 3 | Loss: 5.581608295440674
Epoch: 4 | Loss: 4.9515838623046875
Epoch: 5 | Loss: 4.256173133850098
Epoch: 6 | Loss: 3.4281251430511475
Epoch: 7 | Loss: 2.4838578701019287
Epoch: 8 | Loss: 1.5549306869506836
Epoch: 9 | Loss: 0.7333050966262817
Epoch: 10 | Loss: 0.36228713393211365
